In [3]:
import os

import pandas as pd
import numpy as np

from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from feature_engine import categorical_encoders as ce

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

import matplotlib.pyplot as plt
import seaborn as sns

os.chdir("../src/")
from utils.data_describe import DataDescribe as dd

basepath = "../data/external/"
path_x_train = "../data/external/train_values.csv"
path_y_train = "../data/external/train_labels.csv"
path_x_test = "../data/external/test_values.csv"

path_x_selected = "../data/interim/"
path_processed = "../data/processed/"

# Configurações para mostrar todas as colunas no iPython
# pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [4]:
# List all files in a directory using scandir()
print("-"*30)
print("Arquivos em data/external:\n")
with os.scandir(basepath) as entries:
    for entry in entries:
        if entry.is_file():
            print(entry.name)

print("-"*30)
print("Arquivos em data/interim:\n")

with os.scandir(path_x_selected) as entries:
    for entry in entries:
        if entry.is_file():
            print(entry.name)
print("-"*30)

------------------------------
Arquivos em data/external:

.gitkeep
submission_format.csv
test_values.csv
train_labels.csv
train_values.csv
------------------------------
Arquivos em data/interim:

.gitkeep
lst_X_train_booleanas.csv
lst_X_train_categoricas.csv
lst_X_train_categoricas_nominais.csv
lst_X_train_categoricas_ordinais.csv
lst_X_train_continuas.csv
lst_selected_features.csv
------------------------------


# Início da preparação

A preparação de dados inicial será:
1. One hot encoding dos categóricos nominais;
1. Standard scaling de todos os atributos.

In [5]:
# Carregando dados de treino
lst_selected_features = pd.read_csv(path_x_selected+"lst_selected_features.csv", index_col=0)
lst_selected_features = list(lst_selected_features["0"].unique())

X_train = pd.read_csv(path_x_train, index_col=0)
X_train = X_train[lst_selected_features]

y_train = pd.read_csv(path_y_train, index_col=0)

X_test = pd.read_csv(path_x_test, index_col=0)
X_test = X_test[lst_selected_features]

In [24]:
# Resgatando lista de atributos booleanos e categóricos.
lst_features_rare_label_encoding = pd.read_csv(path_x_selected+"lst_X_train_categoricas_ordinais.csv", index_col=0)
lst_features_rare_label_encoding = list(lst_features_rare_label_encoding["0"].unique())

lst_features_target_encoding = pd.read_csv(path_x_selected+"lst_X_train_categoricas_nominais.csv", index_col=0)
lst_features_target_encoding = list(lst_features_target_encoding["0"].unique())

print(f"""Atributos categóricos ordinais que passarão pelo Rare Label Encoding e posteriormente pelo One Hot Encoding:
{lst_features_rare_label_encoding}

Atributos categóricos nominais que passarão somente por One Hot Encoding:
{lst_features_target_encoding}
""")

Atributos categóricos ordinais que passarão pelo Rare Label Encoding e posteriormente pelo One Hot Encoding:
['count_floors_pre_eq', 'count_families']

Atributos categóricos nominais que passarão somente por One Hot Encoding:
['land_surface_condition', 'roof_type', 'legal_ownership_status', 'other_floor_type', 'position', 'foundation_type', 'ground_floor_type', 'plan_configuration']



## Aplicando o Rare Label Encoding nos atributos categóricos ordinais

In [25]:
# Essa célula, apenans transforma todas as classes dos atributos categoricos ordinais em números inteiros e troca o label 'Rare' para o próximo número inteiro da lista.

def transform_rare_int(df, lst_colunas):
    dct_features = {}

    for feature in lst_colunas:
        lst_features = sorted(df[feature].unique())
        dct_features[feature] = {}
        for feature_in in lst_features:
            if 'Rare' in feature_in:
                dct_features[feature][feature_in] = int(max(dct_features[feature]))+1
            else:
                dct_features[feature][feature_in] = int(feature_in)
        df[feature].replace(dct_features[feature], inplace=True)

    return df

In [26]:
for coluna in X_train[lst_features_rare_label_encoding].columns:
    X_train.loc[:, coluna] = X_train.loc[:, coluna].astype(str)

rare_label_encoder = ce.RareLabelCategoricalEncoder(tol=0.025, n_categories=5,
                                         variables=lst_features_rare_label_encoding,
                                         replace_with='Rare')

X_train = rare_label_encoder.fit_transform(X_train)

X_train = transform_rare_int(X_train, lst_features_rare_label_encoding)

X_train[lst_features_rare_label_encoding].head()

,count_floors_pre_eq,count_families
building_id,,
802906,2,1
28830,2,1
94947,2,1
590882,2,1
201944,3,1


## Aplicando o one hot encoding nos atributos categóricos nominais e ordinais

In [15]:
# Adicionando os atributos que passaram pelo Rare Label Encoding:
lst_features_target_encoding.extend(lst_features_rare_label_encoding)
lst_features_target_encoding = list(set(lst_features_target_encoding))

ohe_hot_encoder = ce.OneHotCategoricalEncoder(top_categories=None,
                                              variables=lst_features_target_encoding,
                                              drop_last=True)

ohe_hot_encoder.fit(X_train)

X_train_target_encoding = ohe_hot_encoder.transform(X_train)

# Criando lista com colunas que passaram pelo OHE:
lst_ohe_encoded_columns = []
for item in lst_features_target_encoding:
    lst_ohe_encoded_columns.extend([coluna for coluna in X_train_target_encoding.columns if item in coluna])

X_train_target_encoding[lst_ohe_encoded_columns].head()

,other_floor_type_q,other_floor_type_x,other_floor_type_j,count_floors_pre_eq_2,count_floors_pre_eq_3,count_floors_pre_eq_1,roof_type_n,roof_type_q,position_t,position_s,position_j,count_families_1,count_families_0,count_families_2,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f,legal_ownership_status_v,legal_ownership_status_a,legal_ownership_status_r,land_surface_condition_t,land_surface_condition_o,ground_floor_type_f,ground_floor_type_x,ground_floor_type_v,ground_floor_type_z,foundation_type_r,foundation_type_w,foundation_type_i,foundation_type_u
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
28830,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0
94947,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
590882,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
201944,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0


In [27]:
# Adicionando os atributos que passaram pelo Rare Label Encoding:
# lst_features_target_encoding.extend(lst_features_rare_label_encoding)
# lst_features_target_encoding = list(set(lst_features_target_encoding))

ohe_hot_encoder = ce.OneHotCategoricalEncoder(top_categories=None,
                                              variables=lst_features_target_encoding,
                                              drop_last=True)

ohe_hot_encoder.fit(X_train)

X_train_target_encoding = ohe_hot_encoder.transform(X_train)

# Criando lista com colunas que passaram pelo OHE:
lst_ohe_encoded_columns = []
for item in lst_features_target_encoding:
    lst_ohe_encoded_columns.extend([coluna for coluna in X_train_target_encoding.columns if item in coluna])

X_train_target_encoding[lst_ohe_encoded_columns].head()

,land_surface_condition_t,land_surface_condition_o,roof_type_n,roof_type_q,legal_ownership_status_v,legal_ownership_status_a,legal_ownership_status_r,other_floor_type_q,other_floor_type_x,other_floor_type_j,position_t,position_s,position_j,foundation_type_r,foundation_type_w,foundation_type_i,foundation_type_u,ground_floor_type_f,ground_floor_type_x,ground_floor_type_v,ground_floor_type_z,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
28830,0,1,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
94947,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
590882,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
201944,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0


## Aplicando Standard Scaler sobre todos os atributos

In [28]:
lst_boolean_features = pd.read_csv(path_x_selected+"lst_X_train_booleanas.csv", index_col=0)
lst_boolean_features = list(lst_boolean_features["0"].unique())
# lst_boolean_features.extend(lst_ohe_encoded_columns)
# lst_boolean_features = list(set(lst_boolean_features))

In [29]:
X_train_target_scaling = X_train_target_encoding.drop(lst_boolean_features, axis=1)
X_train_target_scaling.head()

,count_families,area_percentage,age,geo_level_2_id,geo_level_3_id,height_percentage,geo_level_1_id,count_floors_pre_eq,land_surface_condition_t,land_surface_condition_o,roof_type_n,roof_type_q,legal_ownership_status_v,legal_ownership_status_a,legal_ownership_status_r,other_floor_type_q,other_floor_type_x,other_floor_type_j,position_t,position_s,position_j,foundation_type_r,foundation_type_w,foundation_type_i,foundation_type_u,ground_floor_type_f,ground_floor_type_x,ground_floor_type_v,ground_floor_type_z,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,1,6,30,487,12198,5,6,2,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
28830,1,8,10,900,2812,7,8,2,0,1,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
94947,1,5,10,363,8973,5,21,2,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
590882,1,6,10,418,10694,5,22,2,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
201944,1,8,30,131,1488,9,11,3,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0


In [30]:
scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_target_scaling)
# X_train_scaled = pd.DataFrame(data=X_train_scaled, columns=X_train_target_scaling.columns, index=X_train_target_scaling.index)

X_train_scaled = X_train_target_scaling

# Dataframe para alimentar o modelo

In [31]:
# TEMPORÁRIO

X_train.shape

(260601, 38)

In [32]:
df_train = pd.concat([X_train_scaled, X_train_target_encoding[lst_boolean_features],y_train], axis=1)

df_train.to_parquet(path_processed+"df_train.pqt")

df_train.head()

,count_families,area_percentage,age,geo_level_2_id,geo_level_3_id,height_percentage,geo_level_1_id,count_floors_pre_eq,land_surface_condition_t,land_surface_condition_o,roof_type_n,roof_type_q,legal_ownership_status_v,legal_ownership_status_a,legal_ownership_status_r,other_floor_type_q,other_floor_type_x,other_floor_type_j,position_t,position_s,position_j,foundation_type_r,foundation_type_w,foundation_type_i,foundation_type_u,ground_floor_type_f,ground_floor_type_x,ground_floor_type_v,ground_floor_type_z,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f,has_superstructure_mud_mortar_stone,has_superstructure_timber,has_secondary_use,has_superstructure_adobe_mud,has_superstructure_bamboo,has_superstructure_cement_mortar_brick,has_superstructure_mud_mortar_brick,has_secondary_use_agriculture,has_superstructure_rc_non_engineered,has_superstructure_stone_flag,has_secondary_use_hotel,has_superstructure_cement_mortar_stone,has_superstructure_rc_engineered,has_superstructure_other,has_secondary_use_rental,has_secondary_use_other,has_secondary_use_industry,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
802906,1,6,30,487,12198,5,6,2,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
28830,1,8,10,900,2812,7,8,2,0,1,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
94947,1,5,10,363,8973,5,21,2,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
590882,1,6,10,418,10694,5,22,2,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
201944,1,8,30,131,1488,9,11,3,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [33]:
# Armazenando ordem dos atributos:
lst_features_train = df_train.columns.tolist()
lst_features_train.pop()

'damage_grade'

# Aplicando os encoders treinados no dataset de teste

In [34]:
X_test.head()

,count_families,area_percentage,has_secondary_use_gov_office,has_superstructure_stone_flag,has_superstructure_mud_mortar_stone,has_superstructure_rc_engineered,has_secondary_use,has_superstructure_mud_mortar_brick,position,land_surface_condition,age,geo_level_2_id,geo_level_3_id,height_percentage,has_secondary_use_industry,plan_configuration,has_secondary_use_health_post,has_secondary_use_hotel,has_secondary_use_agriculture,has_superstructure_other,has_superstructure_timber,has_secondary_use_other,has_superstructure_cement_mortar_brick,has_superstructure_cement_mortar_stone,has_secondary_use_rental,foundation_type,has_superstructure_adobe_mud,geo_level_1_id,has_secondary_use_use_police,has_superstructure_rc_non_engineered,has_secondary_use_school,count_floors_pre_eq,ground_floor_type,has_secondary_use_institution,roof_type,has_superstructure_bamboo,legal_ownership_status,other_floor_type
building_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
300051,1,7,0,0,1,0,0,0,s,t,20,596,11307,6,0,d,0,0,0,0,0,0,0,0,0,r,0,17,0,0,0,3,f,0,n,0,v,q
99355,1,13,0,0,1,0,1,0,s,t,25,141,11987,5,0,d,0,0,1,0,0,0,0,0,0,r,0,6,0,0,0,2,f,0,n,0,v,q
890251,1,4,0,0,1,0,0,0,s,t,5,19,10044,5,0,d,0,0,0,0,0,0,0,0,0,r,0,22,0,0,0,2,f,0,n,0,v,q
745817,2,19,0,0,0,0,1,0,t,t,0,39,633,3,0,d,0,0,0,0,0,0,1,0,1,r,0,26,0,0,0,1,v,0,x,0,v,j
421793,1,8,0,0,1,0,0,0,t,t,15,289,7970,7,0,d,0,0,0,0,0,0,0,0,0,r,0,17,0,0,0,3,f,0,q,0,v,q


In [37]:
X_test_encoded = X_test.copy()

X_test_encoded = rare_label_encoder.transform(X_test_encoded)
X_test_encoded = transform_rare_int(X_test_encoded, lst_features_rare_label_encoding)
X_test_ohe_encoded = ohe_hot_encoder.transform(X_test_encoded)
# X_test_encoded_scaled = scaler.transform(X_test_ohe_encoded.drop(lst_boolean_features, axis=1))
X_test_encoded_scaled = X_test_ohe_encoded.drop(lst_boolean_features, axis=1)

columns_encoded_scaled = X_test_ohe_encoded.drop(lst_boolean_features, axis=1)
columns_encoded_scaled = columns_encoded_scaled.columns
index_encoded_scaled = X_test_encoded.index

X_test_ohe_encoded = X_test_ohe_encoded[lst_boolean_features].copy()

X_test_encoded_scaled = pd.DataFrame(data=X_test_encoded_scaled, columns=columns_encoded_scaled,
                                     index=index_encoded_scaled)

X_test_encoded_scaled = pd.concat([X_test_encoded_scaled, X_test_ohe_encoded], axis=1)

In [38]:
X_test_encoded_scaled.to_parquet(path_processed+"X_test_encoded.pqt")